In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import numpy as np

df = pd.read_parquet('./data')
print(f'total dataset: {len(df)} rows')

## filters
# df = df[df['stabilized_spread'] < 10]  # nem tul nagy spread
df = df[df['stabilized_spread'] > 0.3]  # nem tul kis spread

for col in df.columns:
    if 'nr_trades' in col or 'price_delta' in col and not col.startswith('last'):
        df.drop(col, axis=1, inplace=True)
    # if 'nr_trades' in col:
    #     df.drop(col, axis=1, inplace=True)
    if 'past' in col:
        df.drop(col, axis=1, inplace=True)
    if '_spread' == col[1:]:
        df.drop(col, axis=1, inplace=True)
    pass

df_min = df[df.wave_direction == 'min'].copy()
df_max = df[df.wave_direction == 'max'].copy()

df_min.drop('wave_direction', axis=1, inplace=True)
df_max.drop('wave_direction', axis=1, inplace=True)
df = None
print(f'min dataset: {len(df_min)}')
print(f'max dataset: {len(df_max)}')

df_min.sort_index(axis=1, inplace=True)
df_max.sort_index(axis=1, inplace=True)

display(df_max)

# df_max.last_price_delta_since_stabilized = df_max.last_price_delta_since_stabilized * -1
df_max.last_price_delta_since_stabilized.describe()

total dataset: 152280 rows
min dataset: 52879
max dataset: 53800


,0_amount_mean,1_amount_mean,2_amount_mean,3_amount_mean,4_amount_mean,last_price_delta_since_stabilized,stabilized_amount_mean,stabilized_at_ms,stabilized_gasp,stabilized_spread
index,,,,,,,,,,
0,0.0305,0.0815,0.0792,0.0767,0.0725,-0.78,0.0706,108,-0.4798,0.91
0,0.0078,0.0072,0.0122,0.0173,0.0301,-1.84,0.0295,186,0.2329,1.67
0,0.0348,0.0531,0.0503,0.0489,0.0481,-0.74,0.0469,151,-0.8172,1.83
0,0.0031,0.0027,0.0036,0.0058,0.0188,-0.89,0.0167,170,-0.9874,1.81
0,0.0318,0.0315,0.0317,0.0341,0.0360,0.00,0.0345,822,-0.5774,3.55
...,...,...,...,...,...,...,...,...,...,...
0,0.0056,0.0046,0.0051,0.0282,0.0248,-0.19,0.0328,76,-0.9514,0.86
0,0.0234,0.0228,0.0222,0.0211,0.0241,-0.10,0.0280,576,-1.0877,1.71
0,0.0105,0.0473,0.0437,0.0429,0.0424,-0.95,0.0417,88,-0.6657,1.46


count    53800.000000
mean        -0.525841
std          0.928779
min        -17.160000
25%         -0.660000
50%         -0.180000
75%          0.000000
max          0.000000
Name: last_price_delta_since_stabilized, dtype: float64

In [12]:
a = df_min.corr().last_price_delta_since_stabilized * 1000
display(a.sort_values(axis=0, ascending=False))

a = df_max.corr().last_price_delta_since_stabilized * 1000
display(a.sort_values(axis=0, ascending=False))


last_price_delta_since_stabilized    1000.000000
stabilized_spread                     126.387178
stabilized_amount_mean                 69.000968
4_amount_mean                          59.230746
3_amount_mean                          58.248712
2_amount_mean                          53.203914
1_amount_mean                          51.946672
0_amount_mean                          49.915449
stabilized_gasp                        11.456957
stabilized_at_ms                     -284.729249
Name: last_price_delta_since_stabilized, dtype: float64

last_price_delta_since_stabilized    1000.000000
stabilized_at_ms                      282.116124
stabilized_gasp                         4.627223
0_amount_mean                         -43.727080
2_amount_mean                         -46.445248
1_amount_mean                         -47.200144
3_amount_mean                         -51.865044
4_amount_mean                         -55.492567
stabilized_amount_mean                -68.425691
stabilized_spread                    -126.502728
Name: last_price_delta_since_stabilized, dtype: float64

In [13]:
from spreadsurfer.price_engine import FeatureEngineer
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler


def train(df, loss_function):
    # model = CatBoostRegressor(learning_rate=0.01, depth=7, loss_function=loss_function, random_state=0, verbose=False, iterations=1500)
    model = CatBoostRegressor(learning_rate=0.15, depth=6, loss_function=loss_function, random_state=0, verbose=False)
    pipeline = Pipeline(steps=[
        ('preprocessor', FeatureEngineer()),
        ('model', model)
    ])

    df = df.copy()
    y = df['last_price_delta_since_stabilized']
    X = df.drop('last_price_delta_since_stabilized', axis=1)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=0)

    pipeline.fit(X_train, y_train)
    score = pipeline.score(X_valid, y_valid)
    display(score)
    return model, score


# min_model, min_score = train(df_min, loss_function='MAPE')
# max_model, max_score = train(df_max, loss_function='MAPE')

# min_model, min_score = train(df_min, loss_function='RMSE')
# max_model, max_score = train(df_max, loss_function='RMSE')

# min_model, min_score = train(df_min, loss_function='Poisson') # 0.12
# max_model, max_score = train(df_max, loss_function='Poisson') # 0.14

# min_model, min_score = train(df_min, loss_function='Huber:delta=0.4')
# max_model, max_score = train(df_max, loss_function='Huber:delta=0.4')

# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.8')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.7')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.6')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.5')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.4')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.3')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.2')

max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.7')



-0.1733385995985708

In [14]:
from spreadsurfer import now_isoformat

save = True
if save:
    # min_model.save_model(f'./models/{now_isoformat()}-min_-quantile0.7-score-{1000 * round(min_score, 3)}.cat')
    max_model.save_model(f'./models/{now_isoformat()}-max_quantile0.7-score-{1000 * round(max_score, 3)}.cat')
